<a href="https://colab.research.google.com/github/Abhisekh97/RankNet_tf_keras/blob/main/RankNet_tensorflow_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, activations, losses, Model, Input
from tensorflow.nn import leaky_relu
import numpy as np
from itertools import combinations
from tensorflow.keras.utils import plot_model, Progbar
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


##  Model Architecture

In [ ]:
class RankNet(Model):
  def __init__(self):
    super().__init__(self)
    self.dense = [layers.Dense(16, activation=leaky_relu), layers.Dense(8, activation=leaky_relu)]
    self.o = layers.Dense(1, activation='linear')
    self.oi_minus_oj = layers.Subtract()


  def call(self, input):
    xi , xj = input
    densei = self.dense[0](xi)
    densej = self.dense[0](xj)
    for dense in self.dense[1:]:
      densei = dense(densei)
      densej = dense(densej)

    oi = self.o(densei)
    oj = self.o(densej)
    oij = self.oi_minus_oj([oi, oj])
    output = layers.Activation('sigmoid')(oij)
    return output

  def build_graph(self):
    x = [Input(shape=(10)), Input(shape=(10))]
    return Model(inputs=x, outputs=self.call(x))

model = RankNet()
model.build_graph().summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 16)                   176       ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 8)                    136       ['dense[0][0]',           

In [ ]:
nb_query = 20
query = np.array([i+1 for i in range(nb_query) for x in range(int(np.ceil(np.abs(np.random.normal(0,scale=15))+2)))])
query


array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11,
       11, 11, 11, 11, 11

In [ ]:
doc_features = np.random.random((len(query), 10))
doc_features.shape

In [ ]:
doc_scores = np.random.randint(5, size=len(query)).astype(np.float32)
doc_scores.shape

(399,)

In [ ]:
doc_scores

array([4., 2., 3., 0., 0., 4., 3., 3., 1., 4., 2., 3., 2., 0., 3., 1., 2.,
       1., 0., 1., 2., 4., 4., 0., 1., 1., 0., 3., 4., 2., 0., 2., 4., 0.,
       3., 3., 2., 1., 4., 1., 4., 0., 1., 0., 2., 3., 1., 3., 4., 0., 1.,
       3., 1., 1., 3., 1., 3., 0., 2., 3., 0., 3., 1., 0., 0., 3., 3., 2.,
       0., 2., 0., 4., 1., 0., 4., 2., 1., 2., 3., 0., 0., 0., 0., 3., 3.,
       2., 1., 2., 4., 0., 4., 3., 0., 1., 2., 3., 3., 3., 3., 0., 3., 1.,
       4., 0., 2., 2., 1., 3., 2., 2., 4., 3., 0., 4., 1., 1., 0., 3., 4.,
       2., 3., 0., 0., 1., 4., 1., 0., 4., 2., 0., 4., 2., 3., 3., 3., 0.,
       1., 4., 4., 0., 1., 1., 1., 4., 2., 0., 0., 2., 3., 2., 0., 0., 3.,
       4., 4., 3., 0., 1., 0., 0., 3., 0., 4., 2., 3., 4., 2., 2., 3., 0.,
       4., 1., 3., 1., 4., 1., 4., 1., 3., 1., 2., 1., 3., 0., 1., 0., 4.,
       3., 0., 1., 3., 4., 1., 3., 0., 3., 2., 0., 1., 1., 1., 2., 4., 3.,
       3., 0., 1., 3., 2., 3., 4., 0., 2., 0., 4., 2., 0., 2., 4., 4., 1.,
       4., 0., 4., 1., 2.

So above features are doc_features which equals to the number of Query.
so for each query there is one feature vector .
target is basically score provided between 0 to 4 as good to bad relevance. 0 being very bad relevance and 4 being highly relevant items. These labels will be given by Human in practical Scenerio or will be collected from some Human interface as a click Stream Data.  

In [ ]:
xi = []
xj = []
pij = []
pair_id = []
pair_query_id = []
for q in np.unique(query):
  query_idx = np.where(query == q)[0]
  for pair_idx in combinations(query_idx, 2):
    # print(pair_idx)
    pair_query_id.append(q)
    pair_id.append(pair_idx)
    i = pair_idx[0]
    j = pair_idx[1]
    xi.append(doc_features[i])
    xj.append(doc_features[j])

    if doc_scores[i] == doc_scores[j]:
        _pij = 0.5
    elif doc_scores[i] > doc_scores[j]:
        _pij = 1
    else:
        _pij = 0
    pij.append(_pij)


In [ ]:
xi = np.array(xi)
xj = np.array(xj)
pij = np.array(pij)
pair_query_id = np.array(pair_query_id)

In [ ]:
xi_train, xi_test, xj_train, xj_test, pij_train, pij_test, pair_id_train, pair_id_test = train_test_split(
    xi, xj, pij, pair_id, test_size=0.2, stratify=pair_query_id)

In [ ]:
ranknet = RankNet()

In [ ]:
ranknet.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [ ]:
ranknet.fit([xi_train, xj_train], pij_train , epochs=50, batch_size=1, validation_data=([xi_test, xj_test], pij_test))

Epoch 1/50
3984/3984 [==============================] - 12s 2ms/step - loss: 0.6845 - val_loss: 0.6790
Epoch 2/50
3984/3984 [==============================] - 10s 2ms/step - loss: 0.6717 - val_loss: 0.6688
Epoch 3/50
3984/3984 [==============================] - 9s 2ms/step - loss: 0.6596 - val_loss: 0.6555
Epoch 4/50
3984/3984 [==============================] - 9s 2ms/step - loss: 0.6485 - val_loss: 0.6459
Epoch 5/50
3984/3984 [==============================] - 9s 2ms/step - loss: 0.6348 - val_loss: 0.6346
Epoch 6/50
3984/3984 [==============================] - 8s 2ms/step - loss: 0.6215 - val_loss: 0.6292
Epoch 7/50
3984/3984 [==============================] - 9s 2ms/step - loss: 0.6123 - val_loss: 0.6181
Epoch 8/50
3984/3984 [==============================] - 9s 2ms/step - loss: 0.6002 - val_loss: 0.6165
Epoch 9/50
3984/3984 [==============================] - 10s 3ms/step - loss: 0.5926 - val_loss: 0.6094
Epoch 10/50
3984/3984 [==============================] - 9s 2ms/step - loss: 0.